In [ ]:
# ==============================
# 1. Install dependencies
# ==============================
!pip install -q transformers datasets accelerate peft bitsandbytes

In [ ]:
# ==============================
# 2. Import libraries
# ==============================
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from datasets import load_dataset

# Check GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


In [ ]:
# ==============================
# 3. Load model & tokenizer
# ==============================
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

In [ ]:
# ==============================
# 4. Load a free sample dataset
# ==============================
# We'll use the "squad" dataset for QA fine-tuning as an example
dataset = load_dataset("squad", split="train[:1%]")  # small subset for Colab
val_dataset = load_dataset("squad", split="validation[:1%]")

# Preprocess: text -> input_ids, target -> labels
def preprocess(example):
    input_text = example["question"] + " context: " + example["context"]
    target_text = example["answers"]["text"][0] if len(example["answers"]["text"]) > 0 else ""
    model_inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=128)
    labels = tokenizer(target_text, truncation=True, padding="max_length", max_length=32)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train = dataset.map(preprocess, batched=False)
tokenized_val = val_dataset.map(preprocess, batched=False)

In [ ]:

# ==============================
# 5. Setup Training Arguments
# ==============================
training_args = TrainingArguments(
    output_dir="./flan_t5_finetuned",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    learning_rate=5e-5,
    logging_steps=50,
    save_strategy="epoch",

    eval_steps=100,
    save_total_limit=2,
    fp16=True,
    report_to="none",  # disable wandb
)



In [ ]:
# ==============================
# 6. Trainer
# ==============================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)

# ==============================
# 7. Start Training
# ==============================
trainer.train()

# ==============================
# 8. Save model & tokenizer
# ==============================
model.save_pretrained("./flan_t5_finetuned")
tokenizer.save_pretrained("./flan_t5_finetuned")



In [ ]:
# ==============================
# 9. Test inference
# ==============================
test_text = "Who wrote the book 'Pride and Prejudice'? context: Jane Austen wrote the novel 'Pride and Prejudice' in 1813."
inputs = tokenizer(test_text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)
print("Answer:", tokenizer.decode(outputs[0], skip_special_tokens=True))